In [1]:
import laspy
import numpy as np
from scipy.interpolate import griddata
from pyproj import Transformer, CRS
from tqdm import tqdm

In [2]:
# Example usage
laz_file_path = "../lasersurface/lidar_data/3dm_32_356_5645_1_nw.laz"


In [43]:
with laspy.open(laz_file_path) as file:
    las = file.read()

points = np.vstack((las.x, las.y, las.z)).transpose()

ground_points = las.points[las.classification == 2]

In [53]:
gnd_points = np.vstack((ground_points.x, ground_points.y, ground_points.z)).transpose()

In [65]:
geotiff_file = './DEM_data/urn_eop_DLR_CDEM10_Copernicus_DSM_04_N50_00_E006_00_V8239-2020_1__DEM1__coverage_20231204210410.tif'
egm96_geoid_file = './EGM96_data/egm96_15.gtx'
egg08_geoid_file = './EGG08_data/egm08_25.gtx'

In [74]:
transformer = Transformer.from_pipeline(
    f"+proj=pipeline "
    f"+step +inv +proj=utm +zone=32 +ellps=WGS84 "  # Convert from UTM Zone 32N to geographic coordinates
    f"+step +proj=vgridshift +grids={geotiff_file} +multiplier = -1 "  # Vertical grid shift to remove the DEM elevation
    f"+step +proj=vgridshift +grids={egm96_geoid_file} +multiplier = 1 " # Vertical grid shift to add the EGM96 geoid height
    f"+step +proj=vgridshift +grids={egg08_geoid_file} +multiplier = -1 " # Vertical grid shift to remove the EGM2008 geoid height
)

In [76]:
transformed_z = np.array([transformer.transform(xi, yi, zi)[2] for xi, yi, zi in tqdm(zip(las.x, las.y, las.z), total=9471722)])


100%|██████████| 9471722/9471722 [00:38<00:00, 248448.33it/s]


In [75]:
transformer.transform(355890.0,5638399.8,0)

(6.951483912258025, 50.879282150153145, -49.270857730293386)

In [52]:
transformed_z.mean()

2.476752393870345

# test to transform only ground points and check average height

In [46]:
transformed_ground_z = np.array([transformer.transform(xi, yi, zi)[2] for xi, yi, zi in tqdm(zip(ground_points.x, ground_points.y, ground_points.z), total=len(ground_points.x))])

100%|██████████| 3365666/3365666 [00:13<00:00, 255302.60it/s]


In [49]:
transformed_ground_z.mean()

-4.18818157750961

# Check DEM file values with rasterio

In [33]:
import rasterio
from rasterio.transform import from_origin

def get_dem_elevation(dem_file, lat, lon):
    """
    Get the elevation from a DEM file at a given latitude and longitude.

    Parameters:
    dem_file (str): Path to the DEM file.
    lat (float): Latitude of the point.
    lon (float): Longitude of the point.

    Returns:
    float: Elevation at the given latitude and longitude.
    """
    with rasterio.open(dem_file) as dataset:
        # Convert the latitude and longitude to row and column
        row, col = dataset.index(lon, lat)

        # Read the elevation at the given row and column
        elevation = dataset.read(1)[row, col]
        return elevation

In [39]:
latitude, longitude = 50.9392718,6.9659982
elevation = get_dem_elevation(geotiff_file, latitude, longitude)
print(f"Elevation at ({latitude}, {longitude}): {elevation} meters")

Elevation at (50.9392718, 6.9659982): 37.0 meters
